# Implementierung des Gauss Algorithmus

## config and utils:

In [1]:
import numpy as np
from functools import reduce

In [2]:
# we assume the permutations p1, p2 are over the same space 1,....,n
# the i-th entry of an permutation p tells us on which position the i-th row of the matrix sits after the permutation
# example: when n=3 is p1 = [0,1,2] the identity permutation and [2,1,0] switches rows 0 and 2
# we can use reduce in combination with concat_permutation to concat a array of permutations
def concat_permutations(p2, p1):
    result = []
    for i in range(0, len(p1)):
        result.append(p2[p1[i]])
    
    return result
    
def decode_permutation(p):
# to performe the permutation we will decode p into the permutation matrix and perform matrix multiplication
    n = len(p)
    p_ = np.matrix(np.zeros((n, n), dtype=int))

    for i in range(0,n):
        p_[i,p[i]] = 1
    return p_            
            
def permutate_matrix_left(p, a):
    return decode_permutation(p)*a

def permutate_matrix_right(a,p):
    return a*decode_permutation(p)

def permutate_matrix(p,a):
    return permutate_matrix_left(p, permutate_matrix_right(a,p))


## 1. Step: Berechnung der Matrizen P,L,R 

Calculate r and the permutation and elimination matrix for each column

In [3]:
# loop through every column (every inner array)
def compute_r(a, b):#korr: Es wäre besser `DO_PIVOT` und `PRINT_INTERMEDIATE_RESULTS` hier als variable zu setzen
    fs = []
    ps = []
    for i in range(0,n-1):
        # we are inside column i
        if (DO_PIVOT):
            pivot_value = a[i,i]
            pivot_position = i

            # interpret p as a permutation, 
            p = list(range(0,n))
            for j in range(i+1, n):
                if abs(a[j,i]) > abs(pivot_value):
                    # if we have a bigger element change the pivot element and position
                    pivot_value = a[j,i]
                    pivot_position = j
                
            if (pivot_position != i):
                # if we have found a new pivot element switch the rows according to p
                safe = p[i]
                p[i] = p[pivot_position]
                p[pivot_position] = safe
                a = permutate_matrix_left(p, a)
            ps.append(p)
    
        # after we changed the necessary rows and updated our p we can calculate the elimination matrix
        # create a new elimination matrix f_i
        f = np.matrix(np.eye(n))
        for k in range(i+1, n):
            f[k,i] = -(a[k,i] / a[i,i])

        # after we have caluclated the elimination matrix we can calculate the next a
        fs.append(f)
        a = f* a
        if not DO_PIVOT:
            b = f * b
            
    if PRINT_INTERMEDIATE_RESULTS:
        print("R:")
        print(r)
        print("\n")    
    
    return {'r': a, 'fs': fs, 'ps': ps, 'b':b}

## compute p and l

In [4]:
def compute_p_and_l(fs, ps):
    # use reduce to concat all the ps to p
    if DO_PIVOT:
        p = reduce(lambda x, y: concat_permutations(x,y), ps)

        fs_hat = []
        # iterate through fs in reverse order
        for i in range(n-2, -1, -1):
            f = fs[i]
            for j in range(i+1, n-1):
                f = permutate_matrix(ps[j],f)
            fs_hat.append(f)

        # the first element of fs_hat is ^f_n-2
        l = np.matrix(np.eye(n))
        for i in range(0,n-1):
            l = fs_hat[i].getI() * l
    else:
        p = []
        l = []
    if PRINT_INTERMEDIATE_RESULTS and DO_PIVOT:
        print("P:")
        print(p)
        print("\n")

        print("L:")
        print(l)
        print("\n")

        print("PA:")
        print(permutate_matrix_left(p, a_original))
        print("\n")
    
        print("LR:")
        print(l*r)
        print("\n")


    return {'p': p, 'l': l}

## 2. Loesen des Systems Lc = Pb

In [5]:
def solve_lc_pb(l, p, b):
    # define c := Rx = [x'_0, ..... , x'_n-1]
    if DO_PIVOT:
        b_new = permutate_matrix_left(p, b)

        c = np.matrix(np.zeros((n, 1)))

        # use forward substitution to solve lc = b_new
        for i in range(0,n):
            matrix_row_c = 0
            for j in range(0, i):
                matrix_row_c += l[i,j] * c[j]

            c[i] = (b_new[i] - matrix_row_c) / l[i,i]
        
        if PRINT_INTERMEDIATE_RESULTS and DO_PIVOT:
            print("Pb:")
            print(b_new)
            print("\n")

            print("c:")
            print(c)
            print("\n")
        
        return c
    
    if not DO_PIVOT:
        # if we dont use the pivot search mechanismn we still need to assign somethin to c to be able to use the last code snippet
        return b


## 3. Loesen des Systems Rx = c

In [6]:
# with the computed c we can use rx = c using backward substitution
def solve_rx_c(r, c): 
    x = np.matrix(np.zeros((n, 1)))#korr: n kann von r gsetzt werden: n = r.shape[0]

    for i in range(n-1,-1,-1):
        matrix_row_x = 0
        for j in range(i+1,n):
            matrix_row_x += r[i,j] * x[j]
        x[i] = (c[i] - matrix_row_x) / r[i,i]

    return x

## Inputs:

In [ ]:
DO_PIVOT = True
PRINT_INTERMEDIATE_RESULTS = False
pivot = [True, False]

# Testcase a)
print('Testcase a):')
n = 3
xs = []
for j in range(0,2):
    DO_PIVOT = pivot[j]
    a = np.matrix([[1,3,-2],[-2,-5,2],[3,7,-7]], dtype='float64')
    b = np.matrix([[2],[-3],[9]], dtype='float64')

    # the first result contains the matrix r as well as the permutations ps and the elimination matricies fs
    first_result = compute_r(a, b)
    # the second result contains the computed l and p
    second_result = compute_p_and_l(first_result['fs'], first_result['ps'])
    # the third result is the solution c to the system lc = b
    c = solve_lc_pb(second_result['l'], second_result['p'], first_result['b'])
    xs.append(solve_rx_c(first_result['r'], c))
print('with pivot')
print(xs[0])
print('without pivot')
print(xs[1])
print('\n')
print('difference between both methods:')
print(xs[0] - xs[1])
print('\n')
# Testcase b)

print('Testcase b):')
for i in range(10,17):
    xs = []
    for j in range(0,2):
        DO_PIVOT = pivot[j]
        n = i
        a = np.matrix(np.ones((n,n), dtype='float64'))
        np.fill_diagonal(a, 10**(-n))
        b = np.matrix(np.ones((n,1), dtype='float64'))

        first_result = compute_r(a, b)
        second_result = compute_p_and_l(first_result['fs'], first_result['ps'])
        c = solve_lc_pb(second_result['l'], second_result['p'], first_result['b'])
        xs.append(solve_rx_c(first_result['r'], c))
    print(f'solved for n={n}:')
    print('with pivot')
    print(xs[0])
    print('without pivot')
    print(xs[1])
    print('difference between both methods:')
    print(xs[0] - xs[1])
    print('\n')

Testcase a):
[[ 9. ]
 [-1. ]
 [ 2.5]]
[[2.]
 [1.]
 [5.]]
with pivot
[[ 3.]
 [-1.]
 [-1.]]
without pivot
[[ 3.]
 [-1.]
 [-1.]]


difference between both methods:
[[0.]
 [0.]
 [0.]]


Testcase b):
solved for n=10:
with pivot
[[0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]]
without pivot
[[0.11111112]
 [0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]
 [0.11111111]]
difference between both methods:
[[-9.19460907e-09]
 [ 9.87658566e-12]
 [-1.23445698e-12]
 [-1.23449861e-12]
 [-1.23455413e-12]
 [-1.23460964e-12]
 [-1.23467903e-12]
 [-1.23473454e-12]
 [-1.23458188e-12]
 [-1.23456800e-12]]


solved for n=11:
with pivot
[[0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]
 [0.1]]
without pivot
[[0.09999779]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1       ]
 [0.1    

/tmp/ipykernel_116223/1689280415.py:31: RuntimeWarning: invalid value encountered in scalar divide
  f[k,i] = -(a[k,i] / a[i,i])


## Erkenntnisse

bei n=16 scheint bei der Berechnung von R ohne Pivotsuche eine Zahl zu 0 gerundet zu werden, deshalb wird durch 0 geteilt und es kann kein Ergebnis berechnet werden. Daruerberhinaus tritt bei den n zwischen 10 bis 15 

# Korrekturanmerkungen 
+ ~~bei b scheint esw etwas abgehakt~~ habs in der andren dataei gefunden
+ Es ist nicht gut globale variablen in Funktion zu benutzen besser ist sie als Variblen zu Übergeben
  
5/5 Punkten